In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
from torch.utils.data import Dataset, DataLoader

In [3]:
import pandas as pd
import pickle

In [4]:
import torch.utils.data as utils

In [5]:
import numpy as np

In [6]:
from sklearn.preprocessing import LabelEncoder


In [7]:
from sklearn.model_selection import train_test_split

### STEP 2: LOADING DATASET 

In [8]:
class FreeSoundDataset(Dataset):
    """ FreeSound dataset."""

    # Initialize your data, download, etc.
    def __init__(self, X, y):
        
        self.len = X.shape[0]
        self.x_data = torch.from_numpy(X)
        self.y_data = torch.from_numpy(y)

    def __getitem__(self, index):
        return (self.x_data[index], self.y_data[index])

    def __len__(self):
        return self.len

In [9]:
X = np.load('../data/processed/mel/train_curated_mel.npy')

In [10]:
X = X[:,:128,:128]

In [11]:
labels = pd.read_csv('../data/processed/train_curated.csv',sep=';')['labels']

In [12]:
labels = [l.split(',')[0] for l in labels]

In [13]:
le = LabelEncoder()
le.fit(labels)

LabelEncoder()

In [14]:
target = le.transform(labels) 

In [15]:
X_train, X_test, y_train, y_test=train_test_split(X, target, test_size=0.3, random_state=42, stratify=target)

In [16]:
print('X_train:', X_train.shape)

print('X_test:', X_test.shape)

print('y_train:', y_train.shape)
print('y_test:', y_test.shape)

X_train: (3479, 128, 128)
X_test: (1491, 128, 128)
y_train: (3479,)
y_test: (1491,)


In [17]:
train_dataset = FreeSoundDataset(X_train, y_train)

In [18]:
test_dataset = FreeSoundDataset(X_test, y_test)

In [19]:
train_dataset.__getitem__(1)

(tensor([[-0.3540, -0.3531, -0.3558,  ..., -0.0137, -0.2986, -0.3482],
         [-0.4422, -0.4426, -0.4423,  ..., -0.3411, -0.3338, -0.3949],
         [-0.5748, -0.5748, -0.5748,  ..., -0.4962, -0.5112, -0.4954],
         ...,
         [-0.1253, -0.1253, -0.1253,  ..., -0.1246, -0.1246, -0.1244],
         [-0.1362, -0.1363, -0.1362,  ..., -0.1356, -0.1357, -0.1354],
         [-0.1241, -0.1244, -0.1245,  ..., -0.1240, -0.1234, -0.1239]],
        dtype=torch.float64), tensor(62))

### STEP 2: MAKING DATASET ITERABLE

In [20]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
num_epochs

86

In [21]:
transformations = transforms.Compose([transforms.ToTensor()])

In [22]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

In [23]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

### STEP 3: CREATE MODEL CLASS

In [24]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        #self.cnn2 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=1, padding=0)
        #self.relu2 = nn.ReLU()
        
        # Max pool 2
        #self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32 * 63 * 63, 78) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x.float())
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        #out = self.cnn2(out)
        #out = self.relu2(out)
        
        # Max pool 2 
        #out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        out = self.fc1(out)
        
        return out

### STEP 4: INSTANTIATE MODEL CLASS

In [25]:
model = CNNModel()

In [26]:
#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    model.cuda()

### STEP 5: INSTANTIATE LOSS CLASS

In [27]:
criterion = nn.CrossEntropyLoss()

### STEP 6: INSTANTIATE OPTIMIZER CLASS

In [28]:
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### STEP 7: TRAIN THE MODEL

In [30]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.unsqueeze(1).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.unsqueeze(1))
            labels = Variable(labels)
        
        print(images.shape)
        

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Si

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

...

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([79, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Size([100, 1, 128, 198])
torch.Siz

In [29]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.unsqueeze(1).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.unsqueeze(1))
            labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        #images = images.unsqueeze(1).type(torch.FloatTensor).cuda()
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                if torch.cuda.is_available():
                    images = Variable(images.unsqueeze(1).cuda())
                else:
                    images = Variable(images.unsqueeze(1))
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 0.31174468994140625. Accuracy: 17
Iteration: 1000. Loss: 0.12688785791397095. Accuracy: 19
Iteration: 1500. Loss: 0.15483109652996063. Accuracy: 18
Iteration: 2000. Loss: 0.10311100631952286. Accuracy: 19
Iteration: 2500. Loss: 0.034716177731752396. Accuracy: 19
Iteration: 3000. Loss: 0.03624197840690613. Accuracy: 19


In [ ]:
tensor_y = torch.stack([torch.Tensor(i) for i in y])

In [ ]:



my_dataset = utils.TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = utils.DataLoader(my_dataset) # create your dataloader